# Point a

In [67]:
import requests
from key import fred_key
import pandas as pd
import numpy as np
import statsmodels.api as sm



base_url = "https://api.stlouisfed.org/fred/"
obs_endpoint = "series/observations"

start_date = "1950-01-01"
end_date = "2023-12-31"


def create_series_dict(name, series_id):
    return {
        'name': name,
        'series_id': series_id,
        'api_key': fred_key,
        'file_type': 'json',
        'observation_start': start_date,
        'observation_end': end_date,
        'frequency': 'a',
        'units': 'lin'
    }

series_list = [
    ('GDP', 'GDPC1'),
    ('CND', 'PCEND'),
    ('CD', 'PCEDG'),
    ('H', 'HOANBS'),
    ('L', 'PAYEMS'),
    ('AveW', 'AHETPI')
]

request_parameters = [create_series_dict(name, series_id) for name, series_id in series_list]



In [51]:
def fetch_data(parameter):
    response = requests.get(base_url + obs_endpoint, params=parameter)
    if response.status_code == 200:
        res_data = response.json()
        obs_data = pd.DataFrame(res_data['observations'])
        obs_data['date'] = pd.to_datetime(obs_data['date'])
        obs_data.set_index('date', inplace=True)
        obs_data = obs_data.drop(['realtime_start', 'realtime_end'], axis=1)

        # Convert 'value' to numeric, coercing errors to NaN
        obs_data['value'] = pd.to_numeric(obs_data['value'], errors='coerce')

        obs_data.rename(columns={'value': parameter['name']}, inplace=True)
        return obs_data
    else:
        print('Failed to retrieve data. Status code:', response.status_code)
        return pd.DataFrame()  # Return an empty DataFrame on failure
    
    
dataframe = pd.DataFrame()
# concatenate the data
for parameter in request_parameters:
    df = fetch_data(parameter)
    if dataframe.empty:
        dataframe = df
    else:
        dataframe = pd.concat([dataframe, df], axis=1)

# Display the merged dataframe
dataframe

,GDP,CND,CD,H,L,AveW
date,,,,,,
1950-01-01,2458.532,NaN,NaN,42.265,45282,NaN
1951-01-01,2656.320,NaN,NaN,44.261,47926,NaN
1952-01-01,2764.803,NaN,NaN,44.753,48903,NaN
1953-01-01,2894.412,NaN,NaN,45.880,50304,NaN
1954-01-01,2877.708,NaN,NaN,44.331,49087,NaN
...,...,...,...,...,...,...
2019-01-01,20692.087,3006.5,1522.7,102.791,150906,23.51
2020-01-01,20234.074,3084.2,1628.9,95.064,142165,24.69
2021-01-01,21407.693,3500.2,2006.4,100.227,146276,25.91


In [52]:
# adjust the dataset

# add productivity
dataframe['GDP/L'] = (dataframe['GDP']/dataframe['L'])  # annual average income
dataframe['AveH'] = dataframe['H']/dataframe['L']*1000
dataset1 = dataframe.dropna()
# data from 1964 to 2022
dataset1



,GDP,CND,CD,H,L,AveW,GDP/L,AveH
date,,,,,,,,
1964-01-01,4205.277,152.7,59.5,50.351,58394,2.54,0.072016,0.862263
1965-01-01,4478.555,163.3,66.4,52.297,60879,2.63,0.073565,0.859032
1966-01-01,4773.931,177.9,71.8,54.107,64025,2.73,0.074564,0.845092
1967-01-01,4904.864,185.0,74.0,54.113,65935,2.86,0.074389,0.820702
1968-01-01,5145.914,199.8,84.8,55.074,68027,3.02,0.075645,0.809590
1969-01-01,5306.595,214.2,90.5,56.666,70515,3.22,0.075255,0.803602
1970-01-01,5316.391,228.8,90.0,55.799,71007,3.40,0.074871,0.785824
1971-01-01,5491.446,239.7,102.4,55.704,71331,3.63,0.076985,0.780923
1972-01-01,5780.048,257.4,116.4,57.455,73788,3.90,0.078333,0.778650


## Replicate Table 1

In [53]:
sd = dataset1.std()/dataset1.mean()

lags = range(-4, 5)

# initialize a DataFrame to store cross-correlation values
cross_corr_df = pd.DataFrame(index=dataset1.columns, columns=lags)

# calculate cross-correlation for each variable and each lag
for col in dataset1.columns:
    for lag in lags:
        cross_corr_df.at[col, lag] = dataset1['GDP'].corr(dataset1[col].shift(-lag))



cross_corr_df['SD%'] = sd
table1 = cross_corr_df
table1



,-4,-3,-2,-1,0,1,2,3,4,SD%
GDP,0.996157,0.996902,0.998006,0.999012,1.0,0.999012,0.998006,0.996902,0.996157,0.456230
CND,0.989164,0.990058,0.989764,0.987779,0.986553,0.986209,0.98596,0.98637,0.988491,0.750827
CD,0.991568,0.993298,0.989355,0.985254,0.981415,0.978577,0.976664,0.974612,0.974694,0.781369
H,0.956995,0.951185,0.950265,0.951792,0.953955,0.948412,0.940684,0.933495,0.926917,0.206383
L,0.975264,0.971531,0.969728,0.970067,0.971627,0.9699,0.966303,0.962447,0.958619,0.255275
AveW,0.995252,0.995148,0.994857,0.994162,0.993219,0.993012,0.992738,0.992388,0.993354,0.583056
GDP/L,0.988295,0.990481,0.99224,0.99373,0.99417,0.994016,0.995472,0.996699,0.997268,0.226308
AveH,-0.881602,-0.883717,-0.879279,-0.875903,-0.876424,-0.889888,-0.908078,-0.924259,-0.930663,0.065301


In [77]:
# import data for the second table

series_list2 = [
    ('Y', 'A939RC0Q052SBEA'),
    ('C', 'A794RC0Q052SBEA'),
    ('I', 'RGDPLPUSA625NUPN'),
    ('w', 'LES1252881600Q'),
    ('r', 'FEDFUNDS'),
    ('A', 'RTFPNAUSA632NRUG')
]

request_parameters2 = [create_series_dict(name, series_id) for name, series_id in series_list2]


dataframe2 = pd.DataFrame()
# concatenate the data
for parameter in request_parameters2:
    df = fetch_data(parameter)
    if dataframe2.empty:
        dataframe2 = df
    else:
        dataframe2 = pd.concat([dataframe2, df], axis=1)

dataframe2['N'] = dataset1['AveH']
dataframe2['Y/N'] = dataframe2['Y']/dataframe2['N']

dataset2 = dataframe2.dropna()
# data from 1982 to 2007
dataset2 = dataset2.apply(lambda col: np.log(col) if col.name != 'r' else col)



dataset2_c  = pd.DataFrame()
for column in dataset2.columns:
    cycle, _ = sm.tsa.filters.hpfilter(dataset2[column], lamb=1600)
    dataset2_c[column] = cycle

dataset2_c




,Y,C,I,w,r,A,N,Y/N
date,,,,,,,,
1979-01-01,-0.083693,-0.092523,0.036581,0.044840,-1.273928,0.026105,-0.000409,-0.083284
1980-01-01,-0.066321,-0.064115,-0.004881,0.001548,1.415135,0.001684,-0.013425,-0.052896
1981-01-01,-0.015617,-0.033634,-0.011474,-0.017737,4.954995,0.001238,-0.012127,-0.003490
1982-01-01,-0.038160,-0.033650,-0.058464,-0.014830,1.355562,-0.024730,-0.015587,-0.022574
1983-01-01,-0.018503,-0.004745,-0.040804,-0.012009,-1.296347,-0.011789,-0.000705,-0.017798
1984-01-01,0.023957,0.016559,0.001890,-0.012461,0.355234,0.003005,0.013947,0.010010
1985-01-01,0.033452,0.034657,0.012305,0.002801,-1.272917,0.004334,0.010527,0.022925
1986-01-01,0.025441,0.030487,0.015208,0.026955,-2.074244,0.008137,0.000126,0.025316
1987-01-01,0.022715,0.028969,0.015528,0.032365,-1.751395,0.001541,0.006286,0.016429


## Replicate Table 2 

In [83]:
sd2 = dataset2_c.std()
sd_r = sd2/dataset2_c.mean()
p = dataset2_c.apply(lambda series: series.autocorr(lag = 1)) # first order autocorrection (how much the t value is influenced by the t-1 value )
corr_Y = dataset2_c.corr()['Y']

table2 = pd.DataFrame(index=dataset2_c.columns)
table2['SD'] = sd2
table2['SD%'] = sd_r
table2['p'] = p
table2['corr_Y'] = corr_Y
table2


,SD,SD%,p,corr_Y
Y,0.034149,4.167549e+10,0.764941,1.000000
C,0.035240,2.878117e+10,0.827277,0.978454
I,0.031741,2.769267e+10,0.717496,0.518569
w,0.019735,2.266561e+10,0.712322,-0.155843
r,1.802591,3.245313e+12,0.516510,0.142384
A,0.011091,-4.172617e+12,0.558993,0.154666
N,0.012061,-3.608533e+11,0.643295,0.558470
Y/N,0.029182,3.160916e+10,0.789934,0.939387


# Point B